# Distinguish author-specific patterns in music

* Find this notebook at `EpyNN/nnlive/author_music/train.ipynb`.
* Regular python code at `EpyNN/nnlive/author_music/train.py`.

In this notebook we will review:

* Handling univariate time series that represents a **huge amount of data points**.

## Environment and data

Follow [this link](prepare_dataset.ipynb) for details about data preparation.

Briefly, raw data are acoustic guittare music from the *True* author and the *False* author. These are raw ``.wav`` files that were resampled, clipped and digitalized using a 4-bits encoder.

Commonly, music ``.wav`` files have a sampling rate of 44100 Hz. This means that each second of music represents a numerical time series or sequence of length 44100. Because we can not resonably setup a recurrent layer with thousands of cells in the context on this notebook, we dim 

In [1]:
# EpyNN/nnlive/author_music/train.ipynb
# Standard library imports
import random

# Related third party imports
import numpy as np

# Local application/library specific imports
import nnlibs.initialize
from nnlibs.commons.maths import relu, softmax
from nnlibs.commons.library import (
    configure_directory,
    read_model,
)
from nnlibs.network.models import EpyNN
from nnlibs.embedding.models import Embedding
from nnlibs.rnn.models import RNN
# from nnlibs.lstm.models import LSTM
from nnlibs.gru.models import GRU
from nnlibs.flatten.models import Flatten
from nnlibs.dropout.models import Dropout
from nnlibs.dense.models import Dense
from prepare_dataset import prepare_dataset
from settings import se_hPars


########################## CONFIGURE ##########################
random.seed(1)

np.set_printoptions(threshold=10)

np.seterr(all='warn')
np.seterr(under='ignore')


############################ DATASET ##########################
X_features, Y_label = prepare_dataset(N_SAMPLES=10000)

_

## Feed-Forward (FF)

_

### Embedding

_

In [2]:
embedding = Embedding(X_data=X_features,
                      Y_data=Y_label,
                      X_encode=True,
                      Y_encode=True,
                      batch_size=32,
                      relative_size=(2, 1, 0))

_

### Multiple layers with Dropout

_

In [3]:
name = 'Flatten_Dense-64-relu_Dropout-08_Dense-2-softmax'

se_hPars['learning_rate'] = 0.005
se_hPars['softmax_temperature'] = 5

flatten = Flatten()

dropout1 = Dropout(keep_prob=0.5)

hidden_dense = Dense(64, relu)

dropout2 = Dropout(keep_prob=0.5)

dense = Dense(2, softmax)

layers = [embedding, flatten, dropout1, hidden_dense, dropout2, dense]

model = EpyNN(layers=layers, name=name)

_

In [4]:
model.initialize(loss='BCE', seed=1, metrics=['accuracy', 'recall', 'precision'], se_hPars=se_hPars.copy())

--- EpyNN Check --- 
Layer: Embedding
compute_shapes: Embedding
initialize_parameters: Embedding
forward: Embedding
Layer: Flatten
compute_shapes: Flatten
initialize_parameters: Flatten
forward: Flatten
Layer: Dropout
compute_shapes: Dropout
initialize_parameters: Dropout
forward: Dropout
Layer: Dense
compute_shapes: Dense
initialize_parameters: Dense
forward: Dense
Layer: Dropout
compute_shapes: Dropout
initialize_parameters: Dropout
forward: Dropout
Layer: Dense
compute_shapes: Dense
initialize_parameters: Dense
forward: Dense
Layer: Dense
backward: Dense
compute_gradients: Dense
Layer: Dropout
backward: Dropout
compute_gradients: Dropout
Layer: Dense
backward: Dense
compute_gradients: Dense
Layer: Dropout
backward: Dropout
compute_gradients: Dropout
Layer: Flatten
backward: Flatten
compute_gradients: Flatten
Layer: Embedding
backward: Embedding
compute_gradients: Embedding


_

In [5]:
model.train(epochs=100, init_logs=False)

+-------+----------+----------+-------+----------+-------+--------+-------+-----------+-------+-------+-------------------------------------------------------------+
| epoch |  lrate   |  lrate   |       | accuracy |       | recall |       | precision |       |  BCE  |                         Experiment                          |
|       |  Dense   |  Dense   |  (0)  |   (1)    |  (0)  |  (1)   |  (0)  |    (1)    |  (0)  |  (1)  |                                                             |
+-------+----------+----------+-------+----------+-------+--------+-------+-----------+-------+-------+-------------------------------------------------------------+
|   0   | 5.00e-03 | 5.00e-03 | 0.599 |  0.584   | 0.804 | 0.784  | 0.599 |   0.592   | 0.658 | 0.676 | 1628890253_Flatten_Dense-64-relu_Dropout-08_Dense-2-softmax |
|  10   | 5.00e-03 | 5.00e-03 | 0.679 |  0.647   | 0.674 | 0.656  | 0.721 |   0.688   | 0.577 | 0.625 | 1628890253_Flatten_Dense-64-relu_Dropout-08_Dense-2-softmax |
|  2

_

## Recurrent Architectures

_

### Embedding

_

In [6]:
embedding = Embedding(X_data=X_features,
                      Y_data=Y_label,
                      X_encode=True,
                      Y_encode=True,
                      batch_size=32,
                      relative_size=(2, 1, 0))

_

### RNN + Multi-layers FF with Dropout

_

In [7]:
name = 'RNN-100_Flatten_Dense-64-relu_Dropout-08_Dense-2-softmax'

se_hPars['learning_rate'] = 0.001
se_hPars['softmax_temperature'] = 10

rnn = RNN(100, sequences=True)

flatten = Flatten()

dense = Dense(2, softmax)

layers = [embedding, rnn, flatten, dense]

model = EpyNN(layers=layers, name=name)

_

In [8]:
model.initialize(loss='BCE', seed=1, metrics=['accuracy', 'recall', 'precision'], se_hPars=se_hPars.copy())

--- EpyNN Check --- 
Layer: Embedding
compute_shapes: Embedding
initialize_parameters: Embedding
forward: Embedding
Layer: RNN
compute_shapes: RNN
initialize_parameters: RNN
forward: RNN
Layer: Flatten
compute_shapes: Flatten
initialize_parameters: Flatten
forward: Flatten
Layer: Dense
compute_shapes: Dense
initialize_parameters: Dense
forward: Dense
Layer: Dense
backward: Dense
compute_gradients: Dense
Layer: Flatten
backward: Flatten
compute_gradients: Flatten
Layer: RNN
backward: RNN
compute_gradients: RNN
Layer: Embedding
backward: Embedding
compute_gradients: Embedding


_

In [9]:
model.train(epochs=10, init_logs=False)

+-------+----------+----------+-------+----------+-------+--------+-------+-----------+-------+-------+---------------------------------------------------------------------+
| epoch |  lrate   |  lrate   |       | accuracy |       | recall |       | precision |       |  BCE  |                             Experiment                              |
|       |   RNN    |  Dense   |  (0)  |   (1)    |  (0)  |  (1)   |  (0)  |    (1)    |  (0)  |  (1)  |                                                                     |
+-------+----------+----------+-------+----------+-------+--------+-------+-----------+-------+-------+---------------------------------------------------------------------+
|   0   | 1.00e-03 | 1.00e-03 | 0.592 |  0.593   | 0.664 | 0.660  | 0.618 |   0.623   | 0.854 | 0.864 | 1628890370_RNN-100_Flatten_Dense-64-relu_Dropout-08_Dense-2-softmax |
|   1   | 1.00e-03 | 1.00e-03 | 0.613 |  0.604   | 0.645 | 0.635  | 0.647 |   0.642   | 0.774 | 0.802 | 1628890370_RNN-100_Flatten

_

_

### GRU + Multi-layers FF with Dropout

_

In [10]:
name = 'GRU-100_Flatten_Dense-64-relu_Dropout-08_Dense-2-softmax'

se_hPars['learning_rate'] = 0.05
se_hPars['softmax_temperature'] = 10

gru = GRU(100, sequences=True)

flatten = Flatten()

dropout1 = Dropout(keep_prob=0.5)

hidden_dense = Dense(64, relu)

dropout2 = Dropout(keep_prob=0.5)

dense = Dense(2, softmax)

layers = [embedding, gru, flatten, dense]

model = EpyNN(layers=layers, name=name)

_

In [ ]:
model.initialize(loss='BCE', seed=1, metrics=['accuracy', 'recall', 'precision'], se_hPars=se_hPars.copy())

model.train(epochs=10, init_logs=False)

--- EpyNN Check --- 
Layer: Embedding
compute_shapes: Embedding
initialize_parameters: Embedding
forward: Embedding
Layer: GRU
compute_shapes: GRU
initialize_parameters: GRU
forward: GRU
Layer: Flatten
compute_shapes: Flatten
initialize_parameters: Flatten
forward: Flatten
Layer: Dense
compute_shapes: Dense
initialize_parameters: Dense
forward: Dense
Layer: Dense
backward: Dense
compute_gradients: Dense
Layer: Flatten
backward: Flatten
compute_gradients: Flatten
Layer: GRU
backward: GRU
compute_gradients: GRU
Layer: Embedding
backward: Embedding
compute_gradients: Embedding


_

_